# 5 - Multi-class Sentiment Analysis

In all of the previous notebooks we have performed sentiment analysis on a dataset with only two classes, positive or negative. When we have only two classes our output can be a single scalar, bound between 0 and 1, that indicates what class an example belongs to. When we have more than 2 examples, our output must be a $C$ dimensional vector, where $C$ is the number of classes.

In this notebook, we'll be performing classification on a dataset with 6 classes. Note that this dataset isn't actually a sentiment analysis dataset, it's a dataset of questions and the task is to classify what category the question belongs to. However, everything covered in this notebook applies to any dataset with examples that contain an input sequence belonging to one of $C$ classes.

Below, we setup the fields, and load the dataset. 

The first difference is that we do not need to set the `dtype` in the `LABEL` field. When doing a mutli-class problem, PyTorch expects the labels to be numericalized `LongTensor`s. 

The second different is that we use `TREC` instead of `IMDB` to load the `TREC` dataset. The `fine_grained` argument allows us to use the fine-grained labels (of which there are 50 classes) or not (in which case they'll be 6 classes). You can change this how you please.

In [1]:
import torch
from torchtext import data
from torchtext import datasets
import random

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField()

train_data, test_data = datasets.TREC.splits(TEXT, LABEL, fine_grained=False)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

downloading train_5500.label


train_5500.label: 100%|██████████| 336k/336k [00:00<00:00, 451kB/s]


downloading TREC_10.label


TREC_10.label: 100%|██████████| 23.4k/23.4k [00:00<00:00, 126kB/s]


Let's look at one of the examples in the training set.

In [2]:
vars(train_data[-1])

{'label': 'DESC', 'text': ['What', 'is', 'a', 'Cartesian', 'Diver', '?']}

Next, we'll build the vocabulary. As this dataset is small (only ~3800 training examples) it also has a very small vocabulary (~7500 unique tokens), this means we do not need to set a `max_size` on the vocabulary as before.

In [3]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:29, 2.21MB/s]                           
100%|█████████▉| 399623/400000 [00:22<00:00, 17383.31it/s]

Next, we can check the labels.

The 6 labels (for the non-fine-grained case) correspond to the 6 types of questions in the dataset:
- `HUM` for questions about humans
- `ENTY` for questions about entities
- `DESC` for questions asking you for a description 
- `NUM` for questions where the answer is numerical
- `LOC` for questions where the answer is a location
- `ABBR` for questions asking about abbreviations

In [4]:
TEXT.vocab.freqs.most_common(10)

[('?', 3743),
 ('the', 2502),
 ('What', 2265),
 ('is', 1165),
 ('of', 1069),
 ('in', 791),
 ('a', 691),
 ('`', 589),
 ('How', 512),
 ("'s", 494)]

In [5]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f596da6eae8>, {'HUM': 0, 'ENTY': 1, 'DESC': 2, 'NUM': 3, 'LOC': 4, 'ABBR': 5})


As always, we set up the iterators.

In [6]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

We'll be using the CNN model from the previous notebook, however any of the models covered in these tutorials will work on this dataset. The only difference is now the `output_dim` will be $C$ instead of $1$.

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

We define our model, making sure to set `OUTPUT_DIM` to $C$. We can get $C$ easily by using the size of the `LABEL` vocab, much like we used the length of the `TEXT` vocab to get the size of the vocabulary of the input.

The examples in this dataset are generally a lot smaller than those in the IMDb dataset, so we'll use smaller filter sizes.

In [8]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

Checking the number of parameters, we can see how the smaller filter sizes means we have about a third of the parameters than we did for the CNN model on the IMDb dataset.

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 842,406 trainable parameters


Next, we'll load our pre-trained embeddings.

In [10]:
! pip install torchsummaryX

In [11]:
from torchsummaryX import summary
inputs = torch.zeros((100, 1), dtype=torch.long)
summary(model.to(device), inputs.to(device))

100%|█████████▉| 399623/400000 [00:40<00:00, 17383.31it/s]

                      Kernel Shape     Output Shape  Params Mult-Adds
Layer                                                                
0_embedding            [100, 7503]    [1, 100, 100]  750.3k    750.3k
1_convs.Conv2d_0  [1, 100, 2, 100]  [1, 100, 99, 1]   20.1k     1.98M
2_convs.Conv2d_1  [1, 100, 3, 100]  [1, 100, 98, 1]   30.1k     2.94M
3_convs.Conv2d_2  [1, 100, 4, 100]  [1, 100, 97, 1]   40.1k     3.88M
4_dropout                        -         [1, 300]       -         -
5_fc                      [300, 6]           [1, 6]  1.806k      1.8k
------------------------------------------------------------------------
                        Totals
Total params          842.406k
Trainable params      842.406k
Non-trainable params       0.0
Mult-Adds              9.5521M


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_embedding,"[100, 7503]","[1, 100, 100]",750300.0,750300.0
1_convs.Conv2d_0,"[1, 100, 2, 100]","[1, 100, 99, 1]",20100.0,1980000.0
2_convs.Conv2d_1,"[1, 100, 3, 100]","[1, 100, 98, 1]",30100.0,2940000.0
3_convs.Conv2d_2,"[1, 100, 4, 100]","[1, 100, 97, 1]",40100.0,3880000.0
4_dropout,-,"[1, 300]",NaN,NaN
5_fc,"[300, 6]","[1, 6]",1806.0,1800.0


In [12]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [ 0.1638,  0.6046,  1.0789,  ..., -0.3140,  0.1844,  0.3624],
        ...,
        [-0.3110, -0.3398,  1.0308,  ...,  0.5317,  0.2836, -0.0640],
        [ 0.0091,  0.2810,  0.7356,  ..., -0.7508,  0.8967, -0.7631],
        [ 0.4306,  1.2011,  0.0873,  ...,  0.8817,  0.3722,  0.3458]],
       device='cuda:0')

Then zero the initial weights of the unknown and padding tokens.

In [13]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

Another different to the previous notebooks is our loss function (aka criterion). Before we used `BCEWithLogitsLoss`, however now we use `CrossEntropyLoss`. Without going into too much detail, `CrossEntropyLoss` performs a *softmax* function over our model outputs and the loss is given by the *cross entropy* between that and the label.

Generally:
- `CrossEntropyLoss` is used when our examples exclusively belong to one of $C$ classes
- `BCEWithLogitsLoss` is used when our examples exclusively belong to only 2 classes (0 and 1) and is also used in the case where our examples belong to between 0 and $C$ classes (aka multilabel classification).

In [14]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

Before, we had a function that calculated accuracy in the binary label case, where we said if the value was over 0.5 then we would assume it is positive. In the case where we have more than 2 classes, our model outputs a $C$ dimensional vector, where the value of each element is the beleief that the example belongs to that class. 

For example, in our labels we have: 'HUM' = 0, 'ENTY' = 1, 'DESC' = 2, 'NUM' = 3, 'LOC' = 4 and 'ABBR' = 5. If the output of our model was something like: **[5.1, 0.3, 0.1, 2.1, 0.2, 0.6]** this means that the model strongly believes the example belongs to class 0, a question about a human, and slightly believes the example belongs to class 3, a numerical question.

We calculate the accuracy by performing an `argmax` to get the index of the maximum value in the prediction for each element in the batch, and then counting how many times this equals the actual label. We then average this across the batch.

In [15]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]]).to(device)

The training loop is similar to before, without the need to `squeeze` the model predictions as `CrossEntropyLoss` expects the input to be **[batch size, n classes]** and the label to be **[batch size]**.

The label needs to be a `LongTensor`, which it is by default as we did not set the `dtype` to a `FloatTensor` as before.

In [16]:
batch = next(iter(train_iterator))

In [17]:
batch.label

tensor([2, 2, 0, 2, 1, 1, 4, 2, 3, 2, 2, 0, 3, 4, 1, 1, 1, 1, 3, 1, 0, 1, 1, 0,
        1, 2, 0, 3, 2, 1, 4, 4, 1, 2, 2, 0, 4, 2, 1, 0, 2, 0, 0, 0, 2, 0, 4, 2,
        2, 0, 0, 3, 3, 3, 3, 1, 0, 3, 1, 3, 0, 0, 3, 2], device='cuda:0')

In [18]:
batch.text

tensor([[   4,    4,   15,  ...,    4,   38,   43],
        [   5,   19,  603,  ..., 1028,   21,    5],
        [   3,    3,   74,  ...,  725,  709,  884],
        ...,
        [   1,    1,    2,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')

In [19]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

The evaluation loop is, again, similar to before.

In [20]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Next, we train our model.

In [22]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 1.303 | Train Acc: 48.20%
	 Val. Loss: 0.938 |  Val. Acc: 64.65%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.881 | Train Acc: 67.46%
	 Val. Loss: 0.750 |  Val. Acc: 73.79%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.659 | Train Acc: 76.79%
	 Val. Loss: 0.633 |  Val. Acc: 77.36%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.505 | Train Acc: 83.45%
	 Val. Loss: 0.551 |  Val. Acc: 80.68%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.385 | Train Acc: 87.84%
	 Val. Loss: 0.508 |  Val. Acc: 82.20%


Finally, let's run our model on the test set!

In [23]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.423 | Test Acc: 87.09%


Similar to how we made a function to predict sentiment for any given sentences, we can now make a function that will predict the class of question given.

The only difference here is that instead of using a sigmoid function to squash the input between 0 and 1, we use the `argmax` to get the highest predicted class index. We then use this index with the label vocab to get the human readable label.

In [24]:
import spacy
nlp = spacy.load('en')

def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

Now, let's try it out on a few different questions...

In [31]:
type(nlp)

spacy.lang.en.English

In [33]:
sentence = 'how old are you?'

In [34]:
tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
tokenized

['how', 'old', 'are', 'you', '?']

In [35]:
indexed = [TEXT.vocab.stoi[t] for t in tokenized]
indexed

[159, 172, 18, 31, 2]

In [36]:
tensor = torch.LongTensor(indexed).to(device)
tensor

tensor([159, 172,  18,  31,   2], device='cuda:0')

In [37]:
predicted = model(tensor.unsqueeze(1).to('cpu')).squeeze(0)
predicted = F.softmax(predicted)
predicted

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


tensor([0.0974, 0.2791, 0.2137, 0.2484, 0.0909, 0.0705],
       grad_fn=<SoftmaxBackward>)

In [38]:
sorted_values = predicted.argsort(descending=True).cpu().numpy()
sorted_values

array([1, 3, 2, 0, 4, 5])

In [39]:
list(map(lambda x: { "label_idx": x.item(), "label_name": LABEL.vocab.itos[x], 'confidence': predicted[x].item() } , sorted_values))

[{'confidence': 0.27911704778671265, 'label_idx': 1, 'label_name': 'ENTY'},
 {'confidence': 0.24837370216846466, 'label_idx': 3, 'label_name': 'NUM'},
 {'confidence': 0.21371443569660187, 'label_idx': 2, 'label_name': 'DESC'},
 {'confidence': 0.09737875312566757, 'label_idx': 0, 'label_name': 'HUM'},
 {'confidence': 0.09091081470251083, 'label_idx': 4, 'label_name': 'LOC'},
 {'confidence': 0.07050520926713943, 'label_idx': 5, 'label_name': 'ABBR'}]

In [25]:
pred_class = predict_class(model, "Who is Keyser Söze?")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 0 = HUM


In [26]:
pred_class = predict_class(model, "How many minutes are in six hundred and eighteen hours?")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 3 = NUM


In [27]:
pred_class = predict_class(model, "What continent is Bulgaria in?")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 4 = LOC


In [28]:
pred_class = predict_class(model, "What does WYSIWYG stand for?")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 5 = ABBR


## Save the Model

In [29]:
cpu_model = model.to('cpu')
torch.save(cpu_model.state_dict(), 'multiclass_sentiment_analysis.pt')

In [30]:
import pickle

with open('multiclass_sentiment_analysis_metadata.pkl', 'wb') as f:
    metadata = {
        'input_stoi': TEXT.vocab.stoi,
        'label_itos': LABEL.vocab.itos,
        
    }

    pickle.dump(metadata, f)

## Tried with Torch jit

In [40]:
scripted_model = torch.jit.script(model.to('cpu'))

In [41]:
scripted_model(torch.zeros((5, 1), dtype=torch.long))

tensor([[-0.0609,  0.0043,  0.0148, -0.0396,  0.0218, -0.0362]],
       grad_fn=<DifferentiableGraphBackward>)

In [42]:
scripted_model.save('multiclass_sentiment.scripted.pt')

In [47]:
with open("/content/multiclass_sentiment_analysis_metadata.pkl","rb") as vocab_file:
  metadata = pickle.load(vocab_file)
  input_stoi = metadata['input_stoi']
  label_itos = metadata['label_itos']

In [48]:
model = torch.jit.load('/content/multiclass_sentiment.scripted.pt')

In [50]:
import spacy
nlp = spacy.load('en')

def scripted_predict_class(model, sentence, min_len = 4):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [input_stoi[t] for t in tokenized]
    input_tensor = torch.LongTensor(indexed)
    input_tensor = input_tensor.unsqueeze(1)
    with torch.no_grad():
      predicted: Tensor = model(input_tensor)
      predicted = predicted.squeeze(0)
      predicted = F.softmax(predicted)
    sorted_values = predicted.argsort(descending=True).cpu().numpy()
    # preds = model(input_tensor)
    # max_preds = preds.argmax(dim = 1)
    # return max_preds.item()
    toppreds = list(
      map(
          lambda x: {
              "label_idx": x.item(),
              "label_name": label_itos[x],
              "confidence": predicted[x].item(),
          },
          sorted_values,
      )
    )

    return toppreds

In [51]:
prediction = scripted_predict_class(model, "What continent is Bulgaria in?")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [52]:
prediction

[{'confidence': 0.971584141254425, 'label_idx': 4, 'label_name': 'LOC'},
 {'confidence': 0.020134877413511276, 'label_idx': 1, 'label_name': 'ENTY'},
 {'confidence': 0.0030578519217669964, 'label_idx': 0, 'label_name': 'HUM'},
 {'confidence': 0.0025815602857619524, 'label_idx': 3, 'label_name': 'NUM'},
 {'confidence': 0.001791122485883534, 'label_idx': 2, 'label_name': 'DESC'},
 {'confidence': 0.000850483775138855, 'label_idx': 5, 'label_name': 'ABBR'}]